In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import logging
import os 


# Define o nome do arquivo de log
log_filename = 'test_login_results.log'

#Define um diretório para os logs (ex: uma pasta 'logs' no mesmo diretório do script)
log_directory = 'logs'
if not os.path.exists(log_directory):
    os.makedirs(log_directory) # Cria o diretório se ele não existir

# Cria o caminho completo para o arquivo de log
log_filepath = os.path.join(log_directory, log_filename)

# Configura o logging para exibir mensagens de nível INFO ou superior,
# com um formato específico de data/hora, nível e mensagem.
# Adiciona 'filename' para salvar em arquivo e 'filemode='a'' para anexar (append).
# Define 'encoding' para garantir a correta escrita de caracteres especiais.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename=log_filepath, # <--- Adicionado: Nome do arquivo onde os logs serão salvos
    filemode='a',         # <--- Adicionado: Modo de abertura ('a' para append, 'w' para write/sobrescrever)
    encoding='utf-8'      # <--- Adicionado: Garante codificação correta
)

# Ver os logs também no console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logging.getLogger().addHandler(console_handler)
logging.info(f"Logging configurado para console e arquivo: {log_filepath}")


# # Cria uma instância do serviço do ChromeDriver. ChromeDriverManager().install() 
#garante que a versão correta do ChromeDriver seja baixada e utilizada
servico = Service(ChromeDriverManager().install())

# Configurações do Chrome para capturar logs
# Cria uma instância das opções do Chrome.
chrome_options = Options()

# chrome_options.add_argument('--headless')  # Rodar sem abrir o navegador (opcional)

# Adiciona um argumento para desabilitar a aceleração de hardware da GPU, o que pode resolver alguns problemas em ambientes virtuais.
chrome_options.add_argument('--disable-gpu')

# Adiciona um argumento para desabilitar a sandbox do Chrome, necessário em alguns ambientes (como containers).
chrome_options.add_argument('--no-sandbox')

# Adiciona uma opção experimental para habilitar a captura de logs do navegador ('browser': 'ALL').
# NOTA: Estes logs do NAVEGADOR ('browser') ainda serão capturados pelo get_log e registrados pelo logging.info() no arquivo.
chrome_options.add_experimental_option('prefs', {
    'goog:loggingPrefs': {'browser': 'ALL'}
})

# Inicializa uma instância do navegador Chrome com o serviço criado e as opções configuradas
navegador = webdriver.Chrome(service=servico, options=chrome_options)

try:
    # Abre a URL especificada no navegador Chrome.
    navegador.get('http://127.0.0.1:5500/login.html')

    # Registra uma mensagem informativa indicando que a página foi acessada, mostrando a URL atual.
    logging.info(f"Acessando a página: {navegador.current_url}")

    # Arrays com dados de teste para entradas inválidas correspondentes
    emails_errados = [
        "",                     # Campo Nulo
        " ",                    # Campo com espaço
        "usuarioemail.com",     # E-mail faltando @
        "usuario@",             # E-mail faltando o domínio
        "usuario@email",        # E-mail faltando ponto
        "usuario@email",        # E-mail faltando ponto (redundância para acompanhar senhas_erradas)
        "usuario@email"         # E-mail faltando ponto (redundância para acompanhar senhas_erradas)
    ]

    senhas_erradas = [
        "",                     # Campo Nulo
        " ",                    # Campo com espaço
        "1@",                   # Senha com menos de 3 caracteres
        "@Abc",                 # Senha faltando números
        "1Abc",                 # Senha sem caracter especial
        "1@abc",                # Senha sem letra maiúscula
        "1@ABC"                 # Senha sem letra minúscula
    ]


    # Imprime uma mensagem no console para indicar o início do teste com credenciais inválidas, junto com um time para garantir que
    # a mensagem seja exbida no tempo correto.
    print("\n--- Testando Credenciais Inválidas --- (Logs no arquivo)\n")
    time.sleep(1) # Reduzido para 1s

    # Loop para testar os dados de entradas inválidas
    for i in range(len(emails_errados)):
        # Define o número da tentativa atual para fins de log.
        tentativa_num = i + 1

        # Bloco try para capturar possíveis erros durante a interação com a página.
        try:
            # Aguarda até o campo de e-mail estar visível e interativo
            # Espera até 10 segundos até que um elemento com o ID 'email' seja clicável (visível e habilitado).
            campo_email = WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.ID, 'email'))
            )

            # Limpa qualquer texto que possa estar presente no campo de e-mail.
            campo_email.clear()

            # Digita o e-mail inválido atual no campo de e-mail.
            campo_email.send_keys(emails_errados[i])

            # Registra uma mensagem informativa sobre o e-mail digitado na tentativa atual.
            logging.info(f"Tentativa {tentativa_num} (Inválida): E-mail digitado: '{emails_errados[i]}'")

            # Aguarda até o campo de senha estar visível e interativo
            # Espera até 10 segundos até que um elemento com o ID 'password' seja clicável.
            campo_senha = WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.ID, 'password'))
            )

            # Limpa qualquer texto no campo de senha.
            campo_senha.clear()

            # Digita a senha inválida correspondente no campo de senha.
            campo_senha.send_keys(senhas_erradas[i])

            # Registra uma mensagem informativa sobre a senha digitada.
            logging.info(f"Tentativa {tentativa_num} (Inválida): Senha digitada: '{senhas_erradas[i]}'")

            # Aguarda até o botão "Entrar" estar visível
            botao_entrar = WebDriverWait(navegador, 10).until(
                EC.visibility_of_element_located((By.XPATH, "//button[contains(span,'Entrar')]"))
            )
            # Espera até 10 segundos até que um botão contendo o texto 'Entrar' em um span seja visível.


            # Envia TAB e ENTER para acionar o botão
            # Simula pressionar a tecla Tab para focalizar o botão (assumindo que ele está na ordem de foco).
            botao_entrar.send_keys(Keys.TAB)

            # Simula pressionar a tecla Enter para acionar o clique no botão focalizado.
            botao_entrar.send_keys(Keys.ENTER)

            # Registra que as teclas Tab e Enter foram enviadas.
            logging.info(f"Tentativa {tentativa_num} (Inválida): Teclas TAB + ENTER enviadas para o botão 'Entrar'.")

            # Adiciona uma pausa de 2 segundos para dar tempo para a página exibir mensagens de erro ou processar.
            time.sleep(2)

            # Captura os logs do navegador para verificar se há erros ou mensagens relevantes
            browser_logs = navegador.get_log('browser')
            if browser_logs: # Só registra se houver logs do navegador
                logging.info(f"--- Logs do Navegador (Tentativa {tentativa_num} Inválida) ---")
                for entry in browser_logs:
                    # Registra cada log do navegador, incluindo o número da tentativa.
                    logging.info(f"Log Navegador: {entry}")
                logging.info(f"--- Fim Logs do Navegador (Tentativa {tentativa_num} Inválida) ---")

        # Captura qualquer exceção que ocorra dentro do bloco try.
        except Exception as e:
            # Registra o erro ocorrido, incluindo o número da tentativa e a mensagem do erro.
            logging.error(f"Erro na Tentativa {tentativa_num} (Inválida): {e}", exc_info=True) # exc_info=True adiciona traceback


    # Arrays para testar combinações de e-mail e senha válidos e inválidos
    emails_diferentes_validos = [
        "teste@email.com",    # E-mail correto, mas senha válida, porém errada
        "teste@email.com",    # E-mail correto, mas senha inválida
        "errado@email.com",   # E-mail errado, mas senha correta
        "usuario@fake.com",   # E-mail e senha no padrão válido, mas não cadastrado
        "teste@email.com"     # E-mail e senha válidos
    ]

    senhas_diferentes_validas = [
        "1@Ab",               # E-mail correto, mas senha válida, porém errada
        "abc",                # E-mail correto, mas senha inválida
        "1@Bc",               # E-mail errado, mas senha correta
        "2@Cd",               # E-mail e senha no padrão válido, mas não cadastrado
        "1@Bc"                # E-mail e senha válidos
    ]


    # Imprime uma mensagem para indicar o início do segundo conjunto de testes.
    # a mensagem seja exbida no tempo correto.
    print("\n--- Testando Diferentes Combinações de Credenciais --- (Logs no arquivo)\n")
    # logging.info("\n--- Testando Diferentes Combinações de Credenciais ---\n") # Se quiser isso no log também
    time.sleep(1) # Reduzido para 1s

    # Inicia um loop para testar as diferentes combinações de e-mail e senha.
    for i in range(len(emails_diferentes_validos)):
        # Calcula o número da tentativa atual, levando em conta as tentativas anteriores.
        tentativa_num = len(emails_errados) + i + 1

        # Bloco try para capturar erros nesta seção.
        try:
            # Verifica se a URL atual é a de login antes de prosseguir
            # Se o login anterior foi bem-sucedido, precisamos voltar
            if "sucesso.html" in navegador.current_url:
                logging.info(f"Retornando para a página de login de: {navegador.current_url}")
                navegador.back()
                # Aguarda um pouco para a página de login carregar após voltar
                WebDriverWait(navegador, 10).until(EC.url_to_be('http://127.0.0.1:5500/login.html'))
                logging.info(f"Retornou para: {navegador.current_url}")


            # Aguarda até o campo de e-mail estar visível e interativo
            # Espera até que o campo de e-mail seja clicável.
            campo_email = WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.ID, 'email'))
            )

            # Limpa o campo de e-mail.
            campo_email.clear()

            # Digita o e-mail atual.
            campo_email.send_keys(emails_diferentes_validos[i])

            # Registra o e-mail digitado.
            logging.info(f"Tentativa {tentativa_num}: E-mail digitado: '{emails_diferentes_validos[i]}'")

            # Aguarda até o campo de senha estar visível e interativo
            # Espera até que o campo de senha seja clicável.
            campo_senha = WebDriverWait(navegador, 10).until(
                EC.element_to_be_clickable((By.ID, 'password'))
            )

            # Limpa o campo de senha.
            campo_senha.clear()

            # Digita a senha atual.
            campo_senha.send_keys(senhas_diferentes_validas[i])

            # Registra a senha digitada.
            logging.info(f"Tentativa {tentativa_num}: Senha digitada: '{senhas_diferentes_validas[i]}'")

            # Aguarda até o botão "Entrar" estar visível
            botao_entrar = WebDriverWait(navegador, 10).until(
                EC.visibility_of_element_located((By.XPATH, "//button[contains(span,'Entrar')]"))
            )


            # Envia TAB e ENTER para acionar o botão
            # Simula pressionar Tab.
            # Simula pressionar Enter.
            botao_entrar.send_keys(Keys.TAB)
            botao_entrar.send_keys(Keys.ENTER)

            # Registra que as teclas foram enviadas.
            logging.info(f"Tentativa {tentativa_num}: Teclas TAB + ENTER enviadas para o botão 'Entrar'.")

            # Espera explicitamente pelo redirecionamento para a página de sucesso OU falha (fica na mesma página)
            try:
                # Espera até 5 segundos até que a URL da página seja a da página de sucesso.
                WebDriverWait(navegador, 5).until(EC.url_to_be('http://127.0.0.1:5500/sucesso.html'))

                # Registra sucesso e a URL de destino.
                logging.info(f"Tentativa {tentativa_num}: Login bem-sucedido - Redirecionado para {navegador.current_url}")

            except Exception: # TimeoutException é a mais provável, mas Exception pega outras
                # Registra a falha ou a ausência de redirecionamento.
                 # Verifica se a URL ainda é a de login
                if "login.html" in navegador.current_url:
                     logging.warning(f"Tentativa {tentativa_num}: Login FALHOU. Permaneceu em {navegador.current_url}")
                else:
                     logging.warning(f"Tentativa {tentativa_num}: Não redirecionou para sucesso.html em 5s. URL atual: {navegador.current_url}")


            # Captura os logs do navegador para verificar o resultado do login
            browser_logs = navegador.get_log('browser')
            if browser_logs: # Só registra se houver logs do navegador
                logging.info(f"--- Logs do Navegador (Tentativa {tentativa_num}) ---")
                for entry in browser_logs:
                    # Registra cada log, incluindo o número da tentativa.
                    logging.info(f"Log Navegador: {entry}")
                logging.info(f"--- Fim Logs do Navegador (Tentativa {tentativa_num}) ---")

        except Exception as e:
            # Captura erros nesta seção também.
            logging.error(f"Erro na Tentativa {tentativa_num}: {e}", exc_info=True) # exc_info=True adiciona traceback

# Bloco finally que sempre será executado, independentemente de erros.
finally:
    # Adiciona uma pausa de 3 segundo antes de fechar o navegador.
    time.sleep(3)

    # Fecha todas as janelas do navegador e encerra a sessão do WebDriver.
    if 'navegador' in locals() and navegador:
        navegador.quit()

    # Registra que o navegador foi fechado.
    logging.info("Navegador fechado.")
    print("\nExecução concluída. Verifique o arquivo de log:", log_filepath)

2025-05-04 19:41:59,202 - INFO - Logging configurado para console e arquivo: logs\test_login_results.log
2025-05-04 19:41:59,204 - INFO - ====== WebDriver manager ======
2025-05-04 19:42:00,015 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-04 19:42:00,144 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-04 19:42:00,216 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-04 19:42:00,504 - INFO - WebDriver version 136.0.7103.49 selected
2025-05-04 19:42:00,512 - INFO - Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.49/win32/chromedriver-win32.zip
2025-05-04 19:42:00,514 - INFO - About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.49/win32/chromedriver-win32.zip
2025-05-04 19:42:00,714 - INFO - Driver downloading response is 200
2025-05-04 19:42:01,306 - INFO - Get LATEST chromedriver version for google-chrome
2025-05-04 19:42:02,394 - INFO - D


--- Testando Credenciais Inválidas --- (Logs no arquivo)



2025-05-04 19:42:06,793 - INFO - Tentativa 1 (Inválida): E-mail digitado: ''
2025-05-04 19:42:06,854 - INFO - Tentativa 1 (Inválida): Senha digitada: ''
2025-05-04 19:42:06,923 - INFO - Tentativa 1 (Inválida): Teclas TAB + ENTER enviadas para o botão 'Entrar'.
2025-05-04 19:42:08,925 - INFO - --- Logs do Navegador (Tentativa 1 Inválida) ---
2025-05-04 19:42:08,926 - INFO - Log Navegador: {'level': 'SEVERE', 'message': 'http://127.0.0.1:5500/favicon.ico - Failed to load resource: the server responded with a status of 404 (Not Found)', 'source': 'network', 'timestamp': 1746398525673}
2025-05-04 19:42:08,927 - INFO - --- Fim Logs do Navegador (Tentativa 1 Inválida) ---
2025-05-04 19:42:08,982 - INFO - Tentativa 2 (Inválida): E-mail digitado: ' '
2025-05-04 19:42:09,045 - INFO - Tentativa 2 (Inválida): Senha digitada: ' '
2025-05-04 19:42:09,116 - INFO - Tentativa 2 (Inválida): Teclas TAB + ENTER enviadas para o botão 'Entrar'.
2025-05-04 19:42:11,225 - INFO - Tentativa 3 (Inválida): E-mai


--- Testando Diferentes Combinações de Credenciais --- (Logs no arquivo)



2025-05-04 19:42:23,457 - INFO - Tentativa 8: E-mail digitado: 'teste@email.com'
2025-05-04 19:42:23,516 - INFO - Tentativa 8: Senha digitada: '1@Ab'
2025-05-04 19:42:23,586 - INFO - Tentativa 8: Teclas TAB + ENTER enviadas para o botão 'Entrar'.
2025-05-04 19:42:23,616 - WARNING - Tentativa 8: Login FALHOU. Permaneceu em http://127.0.0.1:5500/login.html
2025-05-04 19:42:23,688 - INFO - Tentativa 9: E-mail digitado: 'teste@email.com'
2025-05-04 19:42:23,770 - INFO - Tentativa 9: Senha digitada: 'abc'
2025-05-04 19:42:23,828 - INFO - Tentativa 9: Teclas TAB + ENTER enviadas para o botão 'Entrar'.
2025-05-04 19:42:29,010 - WARNING - Tentativa 9: Login FALHOU. Permaneceu em http://127.0.0.1:5500/login.html
2025-05-04 19:42:29,082 - INFO - Tentativa 10: E-mail digitado: 'errado@email.com'
2025-05-04 19:42:29,149 - INFO - Tentativa 10: Senha digitada: '1@Bc'
2025-05-04 19:42:29,206 - INFO - Tentativa 10: Teclas TAB + ENTER enviadas para o botão 'Entrar'.
2025-05-04 19:42:29,217 - WARNING - 


Execução concluída. Verifique o arquivo de log: logs\test_login_results.log
